In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint
from scipy.stats import invgamma
from matplotlib.lines import Line2D
from scipy.optimize import curve_fit

In [ ]:
def read_in(paths, manhatten_ids):
    dfs = []
    for i,path in enumerate(paths):
        print(str(i+1) + "/" + str(len(paths)))
        data = pd.read_csv(path)[["trip_distance", "PULocationID", "DOLocationID"]]
        # Only take from/to manhatten trips
        data = data[data["PULocationID"].isin(manhatten_ids) & data["DOLocationID"].isin(manhatten_ids)]
        # Convert in km
        data["trip_distance_km"] = data["trip_distance"] * 1.609
        # Only take trips with distance > 0
        data = data[data["trip_distance_km"] > 0]
        dfs.append(data)
        
    return pd.concat(dfs)

In [ ]:
path = "/project.dcf/poss/datasets/NYTaxiData2019"

lookup_df = pd.read_csv(os.path.join(path, "taxi+_zone_lookup.csv"))
manhatten_df = lookup_df[lookup_df["Borough"] == "Manhattan"]
manhatten_ids = manhatten_df["LocationID"].values

files = []
for root,subdirs,subfiles in os.walk(path):
    if "Green" in root or "Yellow" in root:
        for file in subfiles:
            files.append(os.path.join(root, file))

data = read_in(files, manhatten_ids)

In [ ]:
print(len(data))
display(data.head())

In [ ]:
# Take only trips with distance less than 21 km (Manhatten is not max 21 km big)

trip_dists = data[data["trip_distance_km"] < 21]["trip_distance_km"]
# trip_dists = trip_dists/trip_dists.mean()

In [ ]:
trip_dists /= trip_dists.mean()

# take all dists or sample to save time fitting
sample = trip_dists.sample(10000)
# sample = trip_dists
print(len(sample))

In [ ]:
params = invgamma.fit(sample, floc=0, fscale=1.)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(constrained_layout=True)

hist = trip_dists.hist(bins=75, density=True, histtype="step", grid=False, lw=2, ax=ax, color="blue", label="Data")

x = np.linspace(0,trip_dists.max(),1000)
ax.plot(x, invgamma.pdf(x, *params), ls="--", color="red", label="Inverse-gamma Fit")

ax.set_xlabel("Travel Distance (km)")
ax.set_ylabel("Probabiltiy Density")
ax.grid()
ax.legend()

plt.show()

In [ ]:
print(params)
print("k = ", params[0]+1)

In [ ]:
# Try to define custom inv gamma function with constraints on alpa, beta
# Does not work yet

import scipy.stats as st
from scipy.special import gamma

class taxi_dist_gen(st.rv_continuous):

    def _pdf(self, x, k):
        a = k-1
        b = k-2
        return b**a/gamma(a)*x**(-k)*np.exp(-b/x)
#         return invgamma.pdf(x, k-1, 0, k-2)

In [ ]:
taxi_dist = taxi_dist_gen(name='taxi_dist')
taxi_params = taxi_dist.fit(sample)
print(taxi_params)